In [36]:
!pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2gj2kqlz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2gj2kqlz
  Resolved https://github.com/huggingface/transformers.git to commit 59d5edef34ae0fa56065a2e863736d4f133c558b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [37]:
import pandas as pd
df=pd.read_csv('results.csv')
df.head()

,user,title,rating,date,review_text
0,ApTreX,Truly worth the money spent! Even if you go br...,5.0,Reviewed in India 🇮🇳 on 6 February 2022,My honest review after going broke buying this...
1,debarghyo chaudhuri,Review after 1 week of usage,4.0,Reviewed in India 🇮🇳 on 30 July 2022,Got it for 65K on Amazon prime day. So as a li...
2,Pankaj Sangwan,Best product,5.0,Reviewed in India 🇮🇳 on 2 February 2023,I am a regular user of iPhone since iPhone 5s ...
3,Aman Pratap,I own this for gaming but its just become show...,4.0,Reviewed in India 🇮🇳 on 2 February 2023,"Camera quality is better, battery life is very..."
4,Sameer adhate,A great phone,5.0,Reviewed in India 🇮🇳 on 30 January 2023,Everything is good with this phone like camera...


In [38]:
sdf = df[['rating','review_text']]
sdf.head(2)

,rating,review_text
0,5.0,My honest review after going broke buying this...
1,4.0,Got it for 65K on Amazon prime day. So as a li...


In [39]:
def assign_sentiment(rating):
  if float(rating) >= 3:
    return "Positive"
  else:
    return "Negative"
sdf['sentiment'] = sdf['rating'].apply(assign_sentiment)
sdf.head(3)

<ipython-input-39-a7018f5751d7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['sentiment'] = sdf['rating'].apply(assign_sentiment)


,rating,review_text,sentiment
0,5.0,My honest review after going broke buying this...,Positive
1,4.0,Got it for 65K on Amazon prime day. So as a li...,Positive
2,5.0,I am a regular user of iPhone since iPhone 5s ...,Positive


In [40]:
sdf.drop('rating', inplace=True, axis=1)
sdf.head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,review_text,sentiment
0,My honest review after going broke buying this...,Positive
1,Got it for 65K on Amazon prime day. So as a li...,Positive
2,I am a regular user of iPhone since iPhone 5s ...,Positive
3,"Camera quality is better, battery life is very...",Positive
4,Everything is good with this phone like camera...,Positive


In [41]:
from transformers import pipeline
sent_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [42]:
sdf['review_text'] = sdf['review_text'].astype(str)
sdf.head()
sdf = sdf[sdf['review_text'].map(len) <= 2000]
sdf.head()
sdf.dropna(inplace=True)
sdf.reset_index(inplace=True, drop=True)

<ipython-input-42-f6d2f61e2398>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['review_text'] = sdf['review_text'].astype(str)
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [43]:
actual_sentiment = sdf['sentiment']

In [44]:
predicted_sentiment = []
for review in sdf['review_text']:
  predicted_sentiment.append(sent_analysis(review)[0]['label'])

In [45]:
predicted_output = pd.DataFrame(predicted_sentiment)
predicted_output[0] = predicted_output[0].str.title()
predicted_output.head()

,0
0,Positive
1,Positive
2,Negative
3,Positive
4,Negative


In [46]:
actual_output = pd.DataFrame(actual_sentiment)
actual_output.head()

,sentiment
0,Positive
1,Positive
2,Positive
3,Positive
4,Positive


In [47]:
from sklearn.metrics import classification_report
print(classification_report(actual_output['sentiment'], predicted_output[0], target_names=['Negative Reviews','Positive Reviews']))

                  precision    recall  f1-score   support

Negative Reviews       0.00      0.00      0.00         0
Positive Reviews       1.00      0.74      0.85       147

        accuracy                           0.74       147
       macro avg       0.50      0.37      0.43       147
    weighted avg       1.00      0.74      0.85       147



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
